In [24]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import spacy
import re

In [25]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [21]:
%%bigquery products --use_bqstorage_api
SELECT id, name, description, weight, shop_id, price, menu_name, main_category, sub_category
FROM `food-id-app.external_data_temp.EXTERNAL_PRODUCTS`
WHERE main_category = 'Produk Susu'

Downloading: 100%|██████████| 425807/425807 [00:02<00:00, 177257.57rows/s]


In [26]:
skus = []
for i, prod in tqdm(products.iterrows(), total=len(products)):
    sku = re.search(r'(\d){4}/(\d){19}/(\d){2}', prod.description)
    if sku is not None:
        sku = sku.group(0)
        skus.append({
            "id": prod.id,
            "name": prod['name'],
            "description": prod.description,
            "price": prod.price,
            "weight": prod.weight,
            "sku": sku
        })

skus = pd.DataFrame(skus)

100%|██████████| 425807/425807 [00:37<00:00, 11214.34it/s]


In [27]:
skus = skus[skus.sku.duplicated()].reset_index()

In [28]:
skus_list = skus.sku.unique()

In [29]:
skus['master_product'] = skus.apply(lambda x: np.where(skus_list == x.sku)[0][0], axis=1)

In [34]:
one_skus = []
for sku in skus_list:
    num_prods = len(skus[skus.sku == sku])
    if num_prods == 1:
        one_skus.append(sku)

In [39]:
skus = skus[~skus.sku.isin(one_skus)]

In [104]:
skus = skus[skus.sku.isin([sku for sku in skus_list if len(skus[skus.sku == sku]) > 8])].reset_index()

In [105]:
skus.to_csv('../data/sku.csv', index=False)

In [113]:
skus[skus.id == 1229821470].name.iloc[0]

'Cream Cheese / Krim keju Murray Goulburn 2KG Australia Murah'

In [112]:
skus[skus.id == 838709590].name.iloc[0]

'Keju / Cheese Cheddar DairyGold / Dairy Gold Blue 2 KG Keju Parut'